In [1]:
# Cell 1: 导入库
import cv2
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt



In [ ]:
# Cell 2: 定义视频和输出参数
video_path = 'night_scene_1.mp4'
temp_frame_dir = 'temp_frames/'
os.makedirs(temp_frame_dir, exist_ok=True)



In [ ]:


# Cell 3: 将视频解帧到临时文件夹 (如果视频不大，也可以直接在内存里操作)
# -q:v 2 保证解出来的图片质量高
!ffmpeg -i {video_path} -q:v 2 {temp_frame_dir}frame_%04d.png -y

frame_files = sorted([os.path.join(temp_frame_dir, f) for f in os.listdir(temp_frame_dir)])
total_frames = len(frame_files)
print(f"视频总共被分解为 {total_frames} 帧。")



In [ ]:

# Cell 4: 🔥🔥🔥 交互式预览和选择器！🔥🔥🔥
def show_frame(frame_index):
    img = cv2.imread(frame_files[frame_index - 1])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(f'Frame: {frame_index}/{total_frames}')
    plt.axis('off')
    plt.show()

# 创建一个滑动条来浏览所有帧
frame_slider = widgets.IntSlider(min=1, max=total_frames, step=1, value=1, description='预览帧:')
widgets.interact(show_frame, frame_index=frame_slider)

# 创建输入框让你指定想要的帧范围和裁剪区域
start_frame_input = widgets.IntText(value=1, description='起始帧:')
end_frame_input = widgets.IntText(value=8, description='结束帧:')
crop_x = widgets.IntText(value=500, description='裁剪X坐标:')
crop_y = widgets.IntText(value=200, description='裁剪Y坐标:')
crop_size = widgets.IntText(value=512, description='裁剪尺寸:')

display(start_frame_input, end_frame_input, crop_x, crop_y, crop_size)

In [ ]:
# Cell 5: 确认并执行裁剪
process_button = widgets.Button(description="处理并保存")
output_widget = widgets.Output()

def process_frames(b):
    with output_widget:
        clear_output()
        final_input_dir = 'final_input/'
        os.makedirs(final_input_dir, exist_ok=True)
        
        start = start_frame_input.value
        end = end_frame_input.value
        x, y, size = crop_x.value, crop_y.value, crop_size.value

        print(f"将从第{start}帧到第{end}帧，裁剪区域: x={x}, y={y}, size={size}x{size}")

        selected_frames = frame_files[start-1:end]
        for i, frame_path in enumerate(selected_frames):
            img = cv2.imread(frame_path)
            cropped_img = img[y:y+size, x:x+size]
            save_path = os.path.join(final_input_dir, f'input_{i:04d}.png')
            cv2.imwrite(save_path, cropped_img)
        print(f"成功保存 {len(selected_frames)} 帧到 {final_input_dir}")

process_button.on_click(process_frames)
display(process_button, output_widget)